In [ ]:
import pandas as pd
import numpy as np

def calculate_true_range(df):
    df['prev_close'] = df['close_2'].shift(1)
    df['tr1'] = df['high'] - df['low']
    df['tr2'] = abs(df['high'] - df['prev_close'])
    df['tr3'] = abs(df['low'] - df['prev_close'])
    df['TR'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)
    return df

def calculate_directional_movement(df):
    df['+DM'] = np.where((df['high'] - df['high'].shift(1)) > (df['low'].shift(1) - df['low']), 
                         np.where(df['high'] - df['high'].shift(1) > 0, df['high'] - df['high'].shift(1), 0), 0)
    df['-DM'] = np.where((df['low'].shift(1) - df['low']) > (df['high'] - df['high'].shift(1)), 
                         np.where(df['low'].shift(1) - df['low'] > 0, df['low'].shift(1) - df['low'], 0), 0)
    return df

def calculate_smoothed_values(df, period):
    df['TR_smooth'] = df['TR'].rolling(window=period).mean()
    df['+DM_smooth'] = df['+DM'].rolling(window=period).mean()
    df['-DM_smooth'] = df['-DM'].rolling(window=period).mean()
    return df

def calculate_di_adx(df, period, adx_threshold=10, price_action_threshold=0.0001):
    df = calculate_true_range(df)
    df = calculate_directional_movement(df)
    df = calculate_smoothed_values(df, period)

    df['+DI'] = 100 * (df['+DM_smooth'] / df['TR_smooth'])
    df['-DI'] = 100 * (df['-DM_smooth'] / df['TR_smooth'])
    df['DX'] = 100 * abs((df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI']))
    df['ADX'] = df['DX'].rolling(window=period).mean()

    # Generate signals with further simplified conditions
    df['Signal_ADX'] = 0  # Default to no signal
    df.loc[
        (df['+DI'] > 3*df['-DI']) &
        (df['ADX'] > 3*adx_threshold),
        'Signal_ADX'
    ] = 0.2  # Buy Signal
    df.loc[
        (df['-DI'] > 3*df['+DI']) &
        (df['ADX'] > 3*adx_threshold),
        'Signal_ADX'
    ] = -0.2  # Sell Signal

    return df


df = calculate_di_adx(df, period=40)
print(df[['+DI', '-DI', 'ADX', 'Signal_ADX']].head())
